## Defining a Neural Network Model

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [2]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [3]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

for name, param in net.named_parameters():
    if param.requires_grad:
        print(name)

10
torch.Size([6, 1, 5, 5])
conv1.weight
conv1.bias
conv2.weight
conv2.bias
fc1.weight
fc1.bias
fc2.weight
fc2.bias
fc3.weight
fc3.bias


## Forward + Loss

In [4]:
import torch.optim as optim

input = torch.randn(1, 1, 32, 32)  # creating a sample input
output = net(input)  # forward pass
print("output:")
print(output)

net.zero_grad()
output.backward(torch.randn(1, 10))

target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()  # choosing a loss function

loss = criterion(output, target)
print("loss:")
print(loss)

output:
tensor([[ 0.0129,  0.0749,  0.0864,  0.0290,  0.1154,  0.0186,  0.0367,  0.1268,
         -0.0656, -0.0552]], grad_fn=<AddmmBackward0>)
loss:
tensor(0.3106, grad_fn=<MseLossBackward0>)


## Forward + Loss + Backward + Update

In [5]:
import torch.optim as optim

criterion = nn.MSELoss()  # choosing a loss function
optimizer = optim.SGD(net.parameters(), lr=0.01)  # choosing an optimizer

input_data = torch.randn(1, 1, 32, 32)  # creating a sample input data
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output

optimizer.zero_grad()   # zero the gradient buffers

output = net(input_data)  # forward pass

loss = criterion(output, target)

loss.backward()  # backward pass

optimizer.step()  # update parameters